In [ ]:
!pip install transformers fugashi ipadic

In [ ]:
!pip install transformers fugashi ipadic unidic-lite

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
import re

# 日本語BERT（BERT系）モデル
model_name = "cl-tohoku/bert-base-japanese"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# GPU があれば使う
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

def split_sentences(text: str):
    """
    ざっくり句点で文分割（必要ならあとで賢くしてOK）
    """
    sents = re.split(r"(?<=。|\!|\?)", text)
    return [s.strip() for s in sents if s.strip()]

def summarize(text: str, target_chars=120):
    """
    text : 入力テキスト
    target_chars : だいたいこのくらいの文字数がほしい（目安）
    BERTで抽出型要約（重要そうな文を抜き出す）
    """
    sentences = split_sentences(text)
    if not sentences:
        return ""

    # 各文を BERT でベクトルに変換（[CLS]トークンを使う）
    inputs = tokenizer(
        sentences,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        # [batch, seq_len, hidden] -> [batch, hidden]
        sent_embeds = outputs.last_hidden_state[:, 0, :]  # CLS

    # テキスト全体の代表ベクトル（文ベクトルの平均）
    doc_embed = sent_embeds.mean(dim=0, keepdim=True)  # [1, hidden]

    # 各文とのコサイン類似度を計算
    sim = F.cosine_similarity(sent_embeds, doc_embed.repeat(len(sentences), 1))

    # スコアが高い順に文インデックスを並べる
    sorted_idx = torch.argsort(sim, descending=True).tolist()

    # 文字数制限に近づけるように、重要な文から順に追加
    selected = []
    current_len = 0
    max_chars = int(target_chars * 1.3)  # 少し余裕

    for idx in sorted_idx:
        sent = sentences[idx]
        if current_len + len(sent) > max_chars and selected:
            continue
        selected.append((idx, sent))
        current_len += len(sent)
        if current_len >= target_chars:
            break

    # 元の順番に並び替え
    selected.sort(key=lambda x: x[0])
    summary = " ".join(s for _, s in selected)

    return summary

# === 実行部分 ===
text = input("要約したいテキストを貼ってください：\n")
target = int(input("だいたい何文字くらいで要約してほしいですか？："))

print("\n--- 要約結果 (BERT抽出型) ---\n")
result = summarize(text, target_chars=target)
print(result)
print("\n【実際の文字数】：", len(result))